## DICOM Bulk Upload to XNAT
This script is for bulk uploading DICOM files into XNAT project. <br> This script also contains DICOM header editing for XNAT DICOM routing.

In [1]:
import xnat
import pydicom
from pydicom import dcmread
from pydicom.data import get_testdata_file
# import dicom
import zipfile
import os

In [3]:
project_ID = "Melanoma01"
# dir = "\\\\uniwa.uwa.edu.au\\userhome\\staff3\\00105493\\Downloads\\MEL00033"
dir = "C:\\Users\\00105493\\1464_1515"
user = "JulietLi"
password = ""
#Connect to XNAT session
# session = xnat.connect('http://aiqprdeauxnat001.australiaeast.cloudapp.azure.com:8080/xnat-web-1.8.5', user='admin', password='...')
session = xnat.connect('http://instance-146-118-67-25.nimbus.pawsey.org.au', user=user, password=password)


In [3]:
#Function to Zip Directory of Individual Patients
def zip_directory(folder_path, zip_path):
    with zipfile.ZipFile(zip_path, mode='w') as zipf:
        len_dir_path = len(folder_path)
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, file_path[len_dir_path:])

In [49]:
for root, subdirectories, files in os.walk(dir):
    # for subdirectory in subdirectories:
    #     print(os.path.join(root, subdirectory))
    for file in files:
        file_path = os.path.join(root, file)
        ds = pydicom.read_file(file_path)
        #Remove Birthday
        if [0X010, 0X030] in ds:
            del ds[0X010, 0X030]
        #Add patient comments for DICOM routing
        # pc_value = "Project:" + project_ID + " Subject:" + file_path.split('\\')[-3] + " Session:" + file_path.split('\\')[-2] + "_" + str(ds[0X020, 0X011].value)
        pc_value = "Project:" + project_ID + " Subject:" + file_path.split('\\')[-3] + " Session:" + file_path.split('\\')[-2]
        if [0x0010, 0x4000] in ds:
            ds[0x0010, 0x4000].value = pc_value
        else:
            ds.add_new([0x0010, 0x4000], 'LT', pc_value)
        #save dicom file
        ds.save_as(file_path)
        # break


In [50]:
for subdir in os.listdir(dir):
    #check if directory is empty
    count = 0
    for root_dir, sub_dir, files in os.walk(os.path.join(dir, subdir)):
        count += len(files)
    if count > 0:
        print("Zipping: ",os.path.join(dir, subdir))
        zip_directory(os.path.join(dir, subdir), os.path.join(dir, subdir) + ".zip")
        print("Uploading:" + os.path.join(dir, subdir) + ".zip")
        prearchive_session = session.services.import_(os.path.join(dir, subdir) + ".zip", project=project_ID, destination='/prearchive')

Zipping:  C:\Users\00105493\1464_1515\MEL01464


[WARNING] Found unexpect content type, but assuming zip based on the extension


Uploading:C:\Users\00105493\1464_1515\MEL01464.zip
Zipping:  C:\Users\00105493\1464_1515\MEL01482


[WARNING] Found unexpect content type, but assuming zip based on the extension


Uploading:C:\Users\00105493\1464_1515\MEL01482.zip
Zipping:  C:\Users\00105493\1464_1515\MEL01484


...

[WARNING] Found unexpect content type, but assuming zip based on the extension


Uploading:C:\Users\00105493\1464_1515\MEL01514.zip
Zipping:  C:\Users\00105493\1464_1515\MEL01515


[WARNING] Found unexpect content type, but assuming zip based on the extension


Uploading:C:\Users\00105493\1464_1515\MEL01515.zip


Exception in thread XNATpyKeepAliveThread:
Traceback (most recent call last):
  File "c:\Users\00105493\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "c:\Users\00105493\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\util\connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "c:\Users\00105493\AppData\Local\Programs\Python\Python310\lib\socket.py", line 955, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\00105493\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "c:\Users\0

In [4]:
session.disconnect()

In [ ]:
#Show all Projects
# session.projects
# Choose a project
# project_name = session.projects[project_ID]
# #check how many patients
# len(project_name.subjects)

# #Loop through patients
# for subject in project_name.subjects.values():
#     print(subject.label)
#     break





# #Download all patient data
# subject = project_name.subjects['MEL00033']
# subject.download_dir('//uniwa.uwa.edu.au/userhome/staff3/00105493/Downloads/')
# # project.subjects['ANONYMIZ'].experiments['ANONYMIZ'].scans['T'].download('/home/hachterberg/temp/T1.zip')




# #access Prearchive and Archive it
# prearchive_session = session.services.import_('/home/hachterberg/temp/ANONYMIZ.zip', project='brainimages', destination='/prearchive')
# session.prearchive.sessions()
# prearchive_session = session.prearchive.sessions()[0]
# experiment = prearchive_session.archive(subject='ANONYMIZ3', experiment='ANONYMIZ3')